In [8]:
# 必要なPythonライブラリ
import sortedcontainers
from sortedcontainers import SortedDict # ソートされた辞書型データ構造を提供するライブラリ
import random
from sage.misc.sage_timeit import sage_timeit
import time

In [9]:
# 整数をnnビットのバイナリ文字列に変換する
int_to_bin = lambda x, nn: format(x, 'b').zfill(nn)

# バイナリ文字列を再度整数に変換する
def bin_to_int(b):
    out = 0
    bp = b[::-1]
    for i in range(len(bp)):
        # ビット列を逆順にしてから計算
        out += ZZ(bp[i])*2**i
    return out

# nnビットのビットリバーサル置換を行う
def bit_reverse(k, nn):
    bp = int_to_bin(k, nn)[::-1]
    return bin_to_int(bp)

# ベクトルの最初のエントリを範囲1から(q-1)/2に収める
def get_sign(vec):
    i = 0
    while vec[i] == 0 and i < len(vec) - 1:
        i += 1
    if ZZ(vec[i]) > (q - 1) / 2:
        return -1
    else:
        return 1

def sign_assign(vec):
    if get_sign(vec) == -1:
        return [-vec[_] for _ in range(len(vec))]
    else:
        return vec

def is_zero(vec):
    i = 0
    while vec[i] == 0 and i < len(vec) - 1:
        i += 1
    if vec[i] == Mod(0, q):
        return True
    else:
        return False

# ビットリバーサル置換とゼータ操作を行うクラスを作成
class bitZeta():
    def __init__(self, N, B, q):
        self.N = N
        self.nn = 2**N
        # self.kB = kB
        self.B = B
        self.tnn = self.nn * 2
        # ビットリバーサルの結果を格納
        self.bit_reversal_lookup = [bit_reverse(ZZ(Mod(i, self.nn)), self.N) for i in range(self.tnn)]
        # 符号変換テーブルの作成
        self.sign_lookup = [[Mod((-1) ** ((ZZ(Mod(i - h, self.nn)) - (i - h)) / self.nn), q) for i in range(self.nn)] for h in range(self.tnn)]

    # ゼータ操作をプライオリティベースに適用するが、最初のブロックのみに適用する関数
    def bzeta(self, vec, h, ii):
        return [vec[self.bit_reversal_lookup[self.bit_reversal_lookup[i] - h]] * self.sign_lookup[h][self.bit_reversal_lookup[i]] for i in range(self.B * (ii - 1), self.B * ii)]

    ## ビット反転によって位数が高い順にベクトルを並び替えた後、回転を行う
    def zeta(self, vec, h):
        return [vec[self.bit_reversal_lookup[self.bit_reversal_lookup[i] - h]] * self.sign_lookup[h][self.bit_reversal_lookup[i]] for i in range(self.nn)]

In [10]:
# BKW Reductionアルゴリズムの基底クラス
class BKW:

    def __init__(self, q, kn, B, sample_input_list, num_samps, sample_rate_TH, row_RATE):
        self.q = q
        self.kn = kn # 次元
        self.B = B # ブロックサイズ
        self.n = 2**self.kn
        self.nB = len(self.B)
        self.sample_input_list = sample_input_list
        self.num_samps = num_samps
        self.tables = [SortedDict([]) for _ in range(self.nB + 1)] # BKWテーブルの数を定義
        self.passcount = 0 # サンプルの差分が次のテーブルに渡された回数をカウント
        self.kB = log(self.B[self.nB-1], 2)
        self.bit = bitZeta(self.kn, self.kB, self.q) # ビットリバーサルおよびゼータ操作のセットアップ
        alldiffs = False



        #### dynamic block size ####
        self.processed_counter = 0
        self.priority_index = 0 # change_blockによって処理が必要になったサンプルテーブルのインデックスを記録、0なら処理不要
        # for expretiment
        self.sample_rate_TH = sample_rate_TH
        self.row_RATE = row_RATE


        # csv出力よう
        self.total_size = 0



        if alldiffs:
            self.table_insert_blind = self.table_insert_blind_alldiffs
        else:
            self.table_insert_blind = self.table_insert_blind_onediff
        return

    def reduce_final_table(self):
        """
        最終テーブルの重複を取り除くアルゴリズム（ブロック数がステップごとに異なる場合対応）
        chatGPT参照->動いたけど参照したい
        """
        # 最終テーブルを複製
        finaltable = SortedDict([])
        for samp in self.tables[int(self.nB)]:
            finaltable[samp] = self.tables[int(self.nB)][samp]  # finaltableに複製

        # 各サンプルについて処理
        for key in finaltable:
            samp = self.tables[int(self.nB)][key][0]

            # 動的に開始インデックスと終了インデックスを計算
            cumulative_block_sizes = [sum(self.B[:i]) for i in range(1, len(self.B) + 1)]
            start = cumulative_block_sizes[-2]  # 最後から2つ目のブロックの開始位置
            end = cumulative_block_sizes[-1]   # 最終ブロックの終了位置

            # 回転のリストを収集
            rotations = SortedDict([])
            block_size = self.B[-1]  # 現在のステップのブロックサイズ

            for j in range(block_size):
                # 動的に回転量を計算
                rotation_amount = j * block_size
                samp1 = self.bit.zeta(samp, rotation_amount)  # 回転
                samp1abs = sign_assign(samp1)  # 符号を正規化
                rotations[repr(samp1abs[start:end])] = samp1abs

            # 最小の回転結果を選択
            mysamp = rotations.peekitem(0)[1]
            myrep = rotations.peekitem(0)[0]

            # テーブルを更新
            key = repr(samp[start:end])
            if key in self.tables[self.nB]:
                self.tables[int(self.nB)].pop(key)  # 重複を削除
            if myrep not in self.tables[int(self.nB)]:
                self.tables[int(self.nB)][myrep] = [mysamp]  # 最小の正規形を保存

        return

    # 2nd idea: サンプル数に応じてブロック数を変更する
    def change_blocksize(self, i, treating_iteretor, sample_rate_TH, row_RATE):
        '''
        sample_rate_TH = 0.8
        row_RATE = 0.06
        '''
        MAX_ROWS = pow(self.q, self.B[i-1])
        table_rows_TH = MAX_ROWS * row_RATE / 2
        last_index = self.nB - 1

        if (self.num_samps-treating_iteretor)/self.num_samps < sample_rate_TH:
            if len(self.tables[i]) <= table_rows_TH:
                if self.B[i-1] > 1 and i-1 != last_index: # これないと無限に割り続けてしまう.# ここで条件を付け加えることによって最後まで全て分解されてしまうことを防ごう
                  old_block_size = self.B.pop(i-1)

                  #print("changed block s/ize from {} to {}".format(old_block_size, old_block_size/2))

                  self.B.insert(i-1, old_block_size/2)
                  self.B.insert(i-1, old_block_size/2)
                  self.nB = len(self.B)
                  # 元のテーブルを再利用するために初期化する,こいつがiのテーブルになる
                  self.tables[i] = SortedDict([])

                  # 新規用のテーブルを挿入、こいつがi+1のテーブルになる
                  self.tables.insert(i+1, SortedDict([]))
                  # process_counterをリセットしなければならない
                  self.priority_index = i-1
                  self.processed_counter -= len(self.tables[self.priority_index])
                  # print("block array", self.B)
        return


    def report(self):
        # サンプル削減後の基本情報を報告
        self.reduce_final_table()

        # テーブル間のパス数を報告
        print("Number of times a sample was passed to another table:", self.passcount)

        # テーブルサイズを報告
        totalsizes = 0
        for i in range(1, len(self.tables)):
            print("Table", i, "has", len(self.tables[i]), "entries.")
            if i < len(self.tables) - 1:
                totalsizes += len(self.tables[i])
        self.total_size = totalsizes
        print("Total stored table rows (not counting final table):", totalsizes)

        return

    def show_final(self):
        # 最終テーブルを表示
        for i in range(len(self.tables[int(self.nB)])):
            samplist = self.tables[int(self.nB)].peekitem(i)[1]
            for samp in samplist:
                print("           "+str(samp))
    # 1つの差分のみを次に渡すTraditional BKWのテーブル挿入
    def table_insert_blind_onediff(self, samp, i):
        if i == 1:
            # 幾つの初期サンプルを扱ったかのカウント
            self.processed_counter += 1

        # self.change_blocksize(i, self.processed_counter)
        table = self.tables[i]
        samp1 = sign_assign(samp) # 必要に応じて符号を変更
        # 衝突があり、かつ最後のテーブルでない場合は差分を渡す
        start = 0
        for t in range(i-1):
            start += self.B[t]
        rep1 = repr(samp1[start:start + self.B[i-1]])
        if i < self.nB and rep1 in table:
            tabvec = table[rep1][0] # サンプルリストから1つを取り出す
            diff = list(vector(samp1) - vector(tabvec))

            if not is_zero(diff): # 非ゼロの場合、次のテーブルに送る
                self.passcount += 1
                self.table_insert_blind(diff, i + 1)
        else: # 衝突がなく、または最後のテーブルの場合、そのまま格納
            table[rep1] = [samp1] # サンプルをリストとして格納
            self.change_blocksize(i, self.processed_counter, self.sample_rate_TH, self.row_RATE)
        return

In [11]:
# Traditional BKWを完全にリングブラインドで実行（回転を使用しない）
class blind_BKW(BKW):
    def __init__(self, q, kn, B, sample_input_list, num_samps, sample_rate_TH, row_RATE):
        super().__init__(q, kn, B, sample_input_list, num_samps, sample_rate_TH, row_RATE)

    def run(self):
        for s in range(self.num_samps): # 各サンプルを最初のテーブルに渡す
          if self.priority_index != 0:
              # debug用
              prev_samp = None
              for samp in self.tables[self.priority_index]:
                  if samp in self.tables[self.priority_index]:
                      self.table_insert_blind(self.tables[self.priority_index][samp][0], self.priority_index)
                  else:
                      # debug用
                      print("KeyError not found: ", samp)
                      if prev_samp is not None:
                        print("prev samples is ", prev_samp)
                        print(self.tables[self.priority_index][prev_samp][0])
                      return
                  prev_samp = samp
              self.priority_index = 0
          samp = self.sample_input_list[s]
          self.table_insert_blind(samp, 1)

In [12]:
# サンプルとその回転を使用するTraditional BKW
class trad_BKW(BKW):
    def __init__(self, q, kn, B, sample_input_list, num_samps, sample_rate_TH, row_RATE):
        super().__init__(q, kn, B, sample_input_list, num_samps, sample_rate_TH, row_RATE)


    def run(self):
        for s in range(self.num_samps):
            if self.priority_index != 0:
              prev_samp = None # debug用
              for samp in self.tables[self.priority_index]:
                  if samp in self.tables[self.priority_index]:
                      self.table_insert_blind(self.tables[self.priority_index][samp][0], self.priority_index)# ここの第二引数なんでself.priority_index+1じゃなくてself.priority_indexだとうまく行くんだ？
                  else: # この後ろはdebug用
                      print("KeyError not found: ", samp)
                      if prev_samp is not None:
                        print("prev samples is ", prev_samp)
                        print(self.tables[self.priority_index][prev_samp][0])
                      return
                  prev_samp = samp
              self.priority_index = 0
            samp = self.sample_input_list[s] # サンプルそのものの衝突
            self.table_insert_blind(samp, 1)
            for j in range(1, self.n): # サンプルの全回転を最初のテーブルに入力
                samp1 = self.bit.zeta(samp, j)
                self.table_insert_blind(samp1, 1) # 回転させたサンプルの衝突

In [13]:
blindfalse = None
blindtrue = None
tradfalse = None
tradtrue = None

sample_rate_list = []
row_rate_list = []
blind_table_sizes = []
blind_times = []
blind_reduce_samples = []
blind_final_blocks = []
trad_table_sizes = []
trad_times = []
trad_reduce_samples = []
trad_final_blocks = []


def run_experiment(q, kn, B, numsamps, sample_rate_TH, row_RATE, seed=None, show_final=False):
    global blindfalse
    # global blindtrue
    global tradfalse
    # global tradtrue
    global sample_rate_list
    global row_rate_list
    global blind_table_sizes
    global blind_times
    global blind_reduce_samples
    global blind_final_blocks
    global trad_table_sizes
    global trad_times
    global trad_reduce_samples
    global trad_final_blocks


    # パラメータの設定

    n = 2**kn # ベクトルの長さ
    # B = 2**kB # ブロックの長さ

    # 初期サンプルをランダムに生成
    if seed != None:
        seed = int(seed)
        random.seed(seed)
    sample_number = numsamps * n
    sample_input_list = []

    for _ in range(sample_number):
        sample_input_list.append([Mod(random.randint(0, q), q) for _ in range(n)])

    # 各テストを順に実行し、かかった時間と結果を報告

    # Blind BKW with One Diff
    print("*************************************")
    print("Running Blind BKW with One Diff")

    blindfalse = blind_BKW(q, kn, B, sample_input_list, numsamps * n, sample_rate_TH, row_RATE,)

    timed_blindfalse = timeit("blindfalse.run()", number=1, repeat=1, seconds=True)
    print(timed_blindfalse)

    blindfalse.report()

    # Traditional BKW with One Diff
    print("*************************************")
    print("Running Traditional BKW with One Diff")
    tradfalse = trad_BKW(q, kn, B, sample_input_list, numsamps, sample_rate_TH, row_RATE,)
    timed_tradfalse = timeit("tradfalse.run()", number=1, repeat=1, seconds=True)
    print(timed_tradfalse)
    tradfalse.report()


    '''
    #####    ここにcsv書く処理を移行します   #########

    results = [
    float(sample_rate_TH),
    float(row_RATE),

    # blindの格納
    len(blindfalse.tables[-1]),
    timed_blindfalse,
    blindfalse.total_size,
    blindfalse.B,

    # tradの格納
    len(tradfalse.tables[-1]),
    timed_tradfalse,
    tradfalse.total_size,
    tradfalse.B
    ]
    '''
    row_rate_list.append(float(row_RATE))
    sample_rate_list.append(float(sample_rate_TH))
    blind_table_sizes.append(len(blindfalse.tables[-1]))
    blind_times.append(float(timed_blindfalse))
    blind_reduce_samples.append(blindfalse.total_size)
    blind_final_blocks.append(blindfalse.B[-1])

    trad_table_sizes.append(len(tradfalse.tables[-1]))
    trad_times.append(float(timed_tradfalse))
    trad_reduce_samples.append(tradfalse.total_size)
    trad_final_blocks.append(tradfalse.B[-1])
    '''

    import csv

    with open(output_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter='\t')
        if var_param == init_var:
            if fixed_var == Fixed.sample_rate_TH:
                writer.writerow(['sample_rate_TH', 'row_TH', 'blind_table_size', 'blind_time', 'blind_reduce_samples', 'blind_final_blocksize','trad_table_size', 'trad_time', 'trad_reduce_samples', 'trad_final_blocksize'])
            elif fixed_var == Fixed.row_RATE:
                writer.writerow(['sample_rate_TH', 'row_TH', 'blind_table_size', 'blind_time', 'blind_reduce_samples', 'blind_final_blocksize','trad_table_size', 'trad_time', 'trad_reduce_samples', 'trad_final_blocksize'])
            else:
                print("Error: fixed_var is not defined")

        # writer.writerow(results)
    '''





    if show_final:
        tradfalse.show_final()


In [14]:
from enum import Enum

# entries are mod q
q = 17

# dimension of vectors
kn = 4
n = 2^kn #length of vectors

# length of blocks
# kB = 2
# B = 2^kB # length of blocks
len_lastvec = 4
B = [4, 4, 4, len_lastvec]

# number of samples
numsamps = 2000

Fixed = Enum('Fixed', ['sample_rate_TH', 'row_RATE'])

# ここを必要に応じて変更する
fixed_var = Fixed.sample_rate_TH

if fixed_var == Fixed.sample_rate_TH:

    '''
    sample_rate_TH = 0.8
    init_var = 0.01

    end_condition = 0.99
    increment_width = 0.01
    '''
    ## for文のため、整数に調整した値
    sample_rate_TH = 0.8
    init_var = 1

    end_condition = 99
    increment_width = 1

    # file_name = 'sample_rate_TH=' + str(float(sample_rate_TH/100))
elif fixed_var == Fixed.row_RATE:
    '''
    init_var = 0.1
    row_RATE = 0.06

    end_condition = 0.9
    increment_width = 0.1
    '''

    row_RATE = 0.06
    init_var = 10

    end_condition = 90
    increment_width = 1



    # file_name = 'row_RATE=' + str(row_RATE/100)
else:
    print("Error: fixed_var is not defined")

# output_file = '/Users/kenjiro/Documents/2_myj_lab/program/BKWonLWE/experiment_result/fixed_' + file_name + '.csv'

for var_param in range(init_var, end_condition, increment_width):
    stand_var_param = var_param / 100
    if fixed_var == Fixed.sample_rate_TH:
        run_experiment(q,kn,B,numsamps, sample_rate_TH, stand_var_param ,seed=1, show_final=False)
    elif fixed_var == Fixed.row_RATE:
        run_experiment(q,kn,B,numsamps, stand_var_param, row_RATE ,seed=1, show_final=False)
    else:
        print("Error: fixed_var is not defined")
        continue


*************************************
Running Blind BKW with One Diff
17.05909579101717
Number of times a sample was passed to another table: 383232
Table 1 has 9 entries.
Table 2 has 9 entries.
Table 3 has 9 entries.
Table 4 has 9 entries.
Table 5 has 9 entries.
Table 6 has 9 entries.
Table 7 has 9 entries.
Table 8 has 9 entries.
Table 9 has 9 entries.
Table 10 has 9 entries.
Table 11 has 9 entries.
Table 12 has 9 entries.
Table 13 has 9938 entries.
Total stored table rows (not counting final table): 108
*************************************
Running Traditional BKW with One Diff
17.369003750005504
Number of times a sample was passed to another table: 383298
Table 1 has 9 entries.
Table 2 has 9 entries.
Table 3 has 9 entries.
Table 4 has 9 entries.
Table 5 has 9 entries.
Table 6 has 9 entries.
Table 7 has 9 entries.
Table 8 has 9 entries.
Table 9 has 9 entries.
Table 10 has 9 entries.
Table 11 has 9 entries.
Table 12 has 9 entries.
Table 13 has 9926 entries.
Total stored table rows (no

In [15]:
import matplotlib.pyplot as plt

# 保存フォルダ（必要に応じてパスを変更）
output_dir = "/Users/kenjiro/Documents/2_myj_lab/program/BKWonLWE/result_images/"  # 適切なフォルダパスに変更してください

file_prefix= "row_RATE=" + str(row_RATE) if fixed_var == Fixed.row_RATE else "sample_TH" + str(sample_rate_TH)

finename_prefix = output_dir + file_prefix

# 横軸のデータとラベルを設定
x_axis = sample_rate_list if fixed_var == Fixed.row_RATE else row_rate_list
x_label = "Sample Rate" if fixed_var == Fixed.row_RATE else "Row Rate"

# BlindとTradの時間比較グラフ
plt.figure(figsize=(7, 5))
plt.plot(x_axis, blind_times, label="Blind BKW Time", marker='o')
plt.plot(x_axis, trad_times, label="Traditional BKW Time", marker='s')
plt.xlabel(x_label)
plt.ylabel("Time (seconds)")
plt.title("Execution Time")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(f"{finename_prefix}execution_time.png")  # ファイル名を指定
plt.close()

# BlindとTradのテーブルサイズ比較グラフ
plt.figure(figsize=(7, 5))
plt.plot(x_axis, blind_table_sizes, label="Blind Table Size", marker='o')
plt.plot(x_axis, trad_table_sizes, label="Traditional Table Size", marker='s')
plt.xlabel(x_label)
plt.ylabel("Table Size")
plt.title("Final Table Sizes")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(f"{finename_prefix}final_table_sizes.png")  # ファイル名を指定
plt.close()

# BlindとTradのサンプル削減比較グラフ
plt.figure(figsize=(7, 5))
plt.plot(x_axis, blind_reduce_samples, label="Blind Reduced Samples", marker='o')
plt.plot(x_axis, trad_reduce_samples, label="Traditional Reduced Samples", marker='s')
plt.xlabel(x_label)
plt.ylabel("Reduced Samples")
plt.title("Sample Reduction")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig(f"{finename_prefix}sample_reduction.png")  # ファイル名を指定
plt.close()


In [16]:
# グラフを保存
plt.savefig("output_plot.png", dpi=300, bbox_inches='tight')

# グラフを表示
plt.tight_layout()
plt.show()

<Figure size 640x480 with 0 Axes>

In [17]:
print(len(blindfalse.tables[-1]))

9970


In [18]:
print(blindfalse.B)
print(tradfalse.B)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]


In [19]:
print(tradfalse.tables[13])

SortedDict({'[0, 0, 0, 1]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], '[0, 0, 0, 2]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]], '[0, 0, 0, 3]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]], '[0, 0, 0, 4]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]], '[0, 0, 0, 5]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]], '[0, 0, 0, 6]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6]], '[0, 0, 0, 7]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]], '[0, 0, 0, 8]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8]], '[0, 0, 1, 10]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10]], '[0, 0, 1, 11]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11]], '[0, 0, 1, 12]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 12]], '[0, 0, 1, 13]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 13]], '[0, 0, 1, 14]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 14]], '[0, 0, 1, 15]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 15]], '[0, 0, 1, 16]': [[0, 0,

In [20]:
'''
import csv

output_file = '/Users/kenjiro/Documents/2_myj_lab/program/BKWonLWE/tradfalse_finaltable.csv'

# Write SortedDict to CSV
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header row (optional)
    writer.writerow(['Key', 'Values'])

    # Write each key-value pair
    for key, values in blindfalse.tables[10].items():
        for value in values:
            writer.writerow([key, value])

print(f"SortedDict has been written to {output_file}")
'''

'\nimport csv\n\noutput_file = \'/Users/kenjiro/Documents/2_myj_lab/program/BKWonLWE/tradfalse_finaltable.csv\'\n\n# Write SortedDict to CSV\nwith open(output_file, \'w\', newline=\'\') as csvfile:\n    writer = csv.writer(csvfile)\n\n    # Write header row (optional)\n    writer.writerow([\'Key\', \'Values\'])\n\n    # Write each key-value pair\n    for key, values in blindfalse.tables[10].items():\n        for value in values:\n            writer.writerow([key, value])\n\nprint(f"SortedDict has been written to {output_file}")\n'

In [21]:
print("length of tables",len(blindfalse.tables))
print("self.nB", blindfalse.nB)

length of tables 14
self.nB 13
